In [26]:
!pip install psycopg2

In [24]:
# Dependencies

import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy import create_engine

In [14]:
# Import Sea Level csv

csv_file = "epa-sea-level_csv.csv"
sea_level_df = pd.read_csv(csv_file)
sea_level_df.head()

,Year,CSIRO Adjusted Sea Level,Lower Error Bound,Upper Error Bound,NOAA Adjusted Sea Level
0,1880-03-15,0.000000,-0.952756,0.952756,NaN
1,1881-03-15,0.220472,-0.732283,1.173228,NaN
2,1882-03-15,-0.440945,-1.346457,0.464567,NaN
3,1883-03-15,-0.232283,-1.129921,0.665354,NaN
4,1884-03-15,0.590551,-0.283465,1.464567,NaN


In [15]:
# Drop drop columns that will go unused

sea_level_df = sea_level_df[['Year', 'CSIRO Adjusted Sea Level']]
sea_level_df.head()

,Year,CSIRO Adjusted Sea Level
0,1880-03-15,0.000000
1,1881-03-15,0.220472
2,1882-03-15,-0.440945
3,1883-03-15,-0.232283
4,1884-03-15,0.590551


In [16]:
# Strip month and day out of Year column

new_year = []

for index, row in sea_level_df.iterrows():
    new_year.append(int(row[0].split('-')[0]))


sea_level_df['Year'] = new_year

sea_level_df.dropna(how='any', inplace=True)

sea_level_df.tail()

,Year,CSIRO Adjusted Sea Level
129,2009,8.586614
130,2010,8.901575
131,2011,8.964567
132,2012,9.326772
133,2013,8.980315


In [19]:
# Import Global Temp csv

weather_df = pd.read_csv('Global-Temp.csv')
weather_df.head()

,Year,No_Smoothing,Lowess
0,1880,-0.17,-0.09
1,1881,-0.08,-0.13
2,1882,-0.11,-0.17
3,1883,-0.18,0.20
4,1884,-0.28,-0.24


In [20]:
# Drop columns that will go unused

weather_df = weather_df[['Year','No_Smoothing']]
weather_df.head()

,Year,No_Smoothing
0,1880,-0.17
1,1881,-0.08
2,1882,-0.11
3,1883,-0.18
4,1884,-0.28


In [23]:
# Join Dataframes

merged_df = pd.merge(weather_df, sea_level_df, on='Year')

merged_df.head()

,Year,No_Smoothing,CSIRO Adjusted Sea Level
0,1880,-0.17,0.000000
1,1881,-0.08,0.220472
2,1882,-0.11,-0.440945
3,1883,-0.18,-0.232283
4,1884,-0.28,0.590551


In [31]:
# Rename series

merged_df.columns = ['year', 'global_temp', 'sea_level']

merged_df.head()

,year,global_temp,sea_level
0,1880,-0.17,0.000000
1,1881,-0.08,0.220472
2,1882,-0.11,-0.440945
3,1883,-0.18,-0.232283
4,1884,-0.28,0.590551


In [32]:
# Connect to local database

rds_connection_string = 'postgres:changeme@localhost:5432/Project_2'
engine = create_engine(f'postgresql://{rds_connection_string}')



In [33]:
engine.table_names()

['temp_sea_level']

In [34]:
# Use pandas to load csv converted DataFrame into postgres database

merged_df.to_sql(name='temp_sea_level', con=engine, if_exists='append', index=False)